## Import libraries

In [ ]:
import uproot
import awkward as ak
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets

In [ ]:
ctype_to_name = {
    0: 'Undefined',
    1: 'Charged hadron',
    2: 'Electron', 
    3: 'Muon', 
    4: 'Gamma', 
    5: 'Neutral hadron',
    6: 'HF tower, hadron',
    7: 'HF tower, EM',
            }
dm_to_name = {
    0:  '1pr',
    1:  '1pr+1pi0',
    5:  '2pr',
    6:  '2pr+1pi0',
    10: '3pr',
    11: '3pr+1pi0'
}
tau_type_to_name = {
    1: 'e',
    2: "mu",
    3: "tau -> e",
    4: "tau -> mu",
    5: "tau -> h",
    -2147483648: 'not_defined'
}

## Configuration

In [ ]:
N_INNER_CELLS   = 11
INNER_CELL_SIZE = 0.02
N_OUTER_CELLS   = 21
OUTER_CELL_SIZE = 0.05

INNER_LOC = N_INNER_CELLS*INNER_CELL_SIZE/2
OUTER_LOC = N_OUTER_CELLS*OUTER_CELL_SIZE/2

In [ ]:
PATH_TO_FILE = 'data/DYJetsToLL_M-50-amcatnloFXFX_ext2/eventTuple_98.root'
TREE_NAME = 'taus'
EVENT_ID_0 = 33323 # ID of the event to be initially displayed

## Data loading & preprocessing

In [ ]:
with uproot.open(PATH_TO_FILE) as f:
    t = f[TREE_NAME]
    a = t.arrays(['pfCand_pt', 'pfCand_eta', 'pfCand_phi', 'pfCand_particleType',
                  'tau_pt', 'tau_eta', 'tau_phi', 'genLepton_kind',
                  'tau_decayMode', 'tau_decayModeFinding',], how='zip')

In [ ]:
a = a[a['genLepton_kind'] > 0]
a = a[abs(a['tau_phi'])<2*np.pi] # remove candidates with unphysical tau_phi

In [ ]:
# compute dphi and scale it to [-pi, pi]
dphi_array = (a['pfCand', 'phi'] - a['tau_phi'])
dphi_array = np.where(dphi_array <= np.pi, dphi_array, dphi_array - 2*np.pi)
dphi_array = np.where(dphi_array >= -np.pi, dphi_array, dphi_array + 2*np.pi)
a['pfCand', 'dphi'] = dphi_array

# compute deta
a['pfCand', 'deta'] = a['pfCand', 'eta'] - a['tau_eta']

In [ ]:
def compose_tau_data(a, event_id):
    c_type = list(map(ctype_to_name.get, a['pfCand', 'particleType'][event_id]))
    tau_type = tau_type_to_name[a['genLepton_kind'][event_id]]
    tau_DM = dm_to_name[a['tau_decayMode'][event_id]]
    tau_pt = a['tau_pt'][event_id]
    tau_df = pd.DataFrame({'deta': a['pfCand', 'deta'][event_id],
                   'dphi': a['pfCand', 'dphi'][event_id],
                   'pt': a['pfCand', 'pt'][event_id],
                   'DM': a['tau_decayMode'][event_id],
                   'type': c_type})
    return tau_df, tau_pt, tau_type, tau_DM

## Plot single figure

In [ ]:
def create_figure(a, event_id):
    tau_df, tau_pt, tau_type, tau_DM = compose_tau_data(a, event_id)
    fig = px.scatter(tau_df, x="deta", y="dphi", color="type",
                 size='pt', hover_data=['pt', 'deta', 'dphi'],
                 title=f'gen type: {tau_type};   pt: {tau_pt: .1f} GeV;   reco DM: {tau_DM}') # ;   event ID: {event_id}
    fig.add_shape(type="rect",
        x0=-INNER_LOC, y0=-INNER_LOC, x1=INNER_LOC, y1=INNER_LOC,
        line=dict(color="RoyalBlue"),
    )
    fig.add_shape(type="rect",
        x0=-OUTER_LOC, y0=-OUTER_LOC, x1=OUTER_LOC, y1=OUTER_LOC,
        line=dict(color="LightCoral"),
    )
    fig.update_layout(autosize=False,
        width=650,
        height=650,           
    )  
    fig.update_xaxes(range=[-0.8, 0.8])
    fig.update_yaxes(range=[-0.8, 0.8])
    return fig

In [ ]:
fig = create_figure(a, EVENT_ID_0)
fig.show()

## Build widget

In [ ]:
# Assign figure to an empty widget
g = go.FigureWidget(data=fig,
                    layout=go.Layout(
                        title=dict(
                            text='Widget'
                        ),
                    ))

In [ ]:
event_text = widgets.IntText(EVENT_ID_0) # switcher for event ID
container = widgets.HBox(children=[event_text])

In [ ]:
def response(change):  
    event_id = event_text.value
    if event_id < len(a) and event_id > -1:
        tau_df, tau_pt, tau_type, tau_DM = compose_tau_data(a, event_id)
        fig = px.scatter(tau_df, x="deta", y="dphi", color="type",
                         size='pt', hover_data=['pt', 'deta', 'dphi'],)
        with g.batch_update():
            g.data = ()
            [g.add_traces(item) for item in fig.data]
            g.layout.title.text = f'gen type: {tau_type};   reco pt: {tau_pt: .1f} GeV;   reco DM: {tau_DM}'
            
event_text.observe(response, names="value")

In [ ]:
widgets.VBox([container, g])